Chapter 2.a.iii.  ExprRange reductions
=======

As we saw in the previous chapter, an **ExprRange** adds versatility to **Prove-It** by allowing us to define **Lambda** expressions with an indeterminant number of parameters.  Generally, **ExprRange** expressions allow us to form expressions over an indeterminant number of elements.  Basically, an **ExprRange** is how we formalize the concept of using an ellipsis in a mathematical expression.  For example, $a \cdot (b_1 + \ldots + b_n) = a \cdot b_1 + \ldots + a \cdot b_n$ (distribution) may be expressed using an **ExprRange** on each side of the equation.  Internally, the **ExprRange** has a *parameter* variable for parameterizing each element of the range.  For the LaTeX display, this *parameter* is hidden by default, as in the above example, and the precise parameterization is implicit.  To avoid ambiguity, it can be important at times to display the explicit parameterization.  For the above example, this would be $a \cdot (b_1 + ..b_k.. + b_n) = a \cdot b_1 + ..a \cdot b_k.. + a \cdot b_n$, assuming $k$ is the *parameter* in both **ExprRange** instances.

There are two kinds of reduction rules that are specific to **ExprRange** expressions that are applied in the process of performing a lambda application (beta reduction).
1. *Indexed variable expansions* occur when the **ExprRange** contains one or more **IndexedVar** expressions with an index that is parameterized over the range, and the range of indexed variables is being replaced by an **ExprTuple**.  This falls under two cases.
  1. *Parameter dependent expansion* is required whenever the parameter of the **ExprRange** occurs anywhere besides as an index of an indexed variable.  In that case, the indices of the expanded indexed variable must match the original indices.  For example, $1 \cdot x_1 + \ldots + n \cdot x_n$ could be expanded, under $(x_1, \ldots, x_n) : (a_1, \ldots, a_j, a_{j+1}, \ldots, a_n)$ to $1 \cdot a_1 + \ldots + j \cdot a_j + (j+1) \cdot a_{j+1} + \ldots + n \cdot a_n$ assuming $0 \leq j \leq n$.  However, an error would occur given $(x_1, \ldots, x_n) : (a_1, \ldots, a_j, b_1, \ldots, b_k)$ as the expansion.
  2. *Parameter independent expansion* is allowed otherwise.  In that case, only the lengths of the expansion must match, not the indices themselves.  For example, $x_1 \cdot y_1 + \ldots + x_n \cdot y_n$ could be expanded, under $(x_1, \ldots, x_n) : (a_1, \ldots, a_j, b_1, \ldots, b_k)$, $(y_1, \ldots, y_n) : (c_1, \ldots, c_j, d_1, \ldots, d_k)$ to $a_1 \cdot c_1 + \ldots + a_j \cdot c_j + b_1 \cdot d_1 + \ldots + b_k \cdot d_k$ assuming $j + k = n$.
2. *Range reductions* occur when the **ExprRange** is known to be empty or contain only a single element. 
  1. *Empty range reduction* occurs when the **ExprRange** is known to be empty and may be replaced with zero elements in a containing **ExprTuple**.
  2. *Singular element range reduction* occurs when the **ExprRange** is known to contain only a single element and may be replaced with this one element.

In [ ]:
import proveit
%begin exprrange_reductions

## Creating ExprRange expressions

An `ExprRange` may be instantiated with a `parameter`, `body`, `start index`, and `end index`:

In [ ]:
from proveit import ExprTuple, ExprRange, defaults
from proveit.number import Add, Mult, zero, one, two
from proveit._common_ import a, b, c, d, i, j, k, m, n, x, y
a_prod_range = ExprRange(k, Mult(a, k), i, j)

In [ ]:
a_prod_range.parameter

In [ ]:
a_prod_range.body

In [ ]:
a_prod_range.start_index

In [ ]:
a_prod_range.end_index

It's sub-expressions are a **Lambda** expression created from the `parameter` and `body`, the `start_index`, and the `end_index`:

In [ ]:
a_prod_range.exprInfo()

This `lambda_map` may be accessed directly:

In [ ]:
a_prod_range.lambda_map

You can create the same **ExprRange** by proving `None` for the `parameter` and `body` but suppling the `lambda_map` instead:

In [ ]:
from proveit import Lambda
a_prod_range_again = ExprRange(None, None, i, j, lambda_map=Lambda(k, Mult(a, k)))

In [ ]:
a_prod_range_again == a_prod_range

When **ExprRange**(s) are used as *operands* of an **Operation**, it is typically formatted in a clean manner that reflects that there are an indeterminant number of *operands*:

In [ ]:
add_with_ranges = Add(a_prod_range, b, a_prod_range)

Note that the *operands* are an **ExprTuple** that contains the **ExprRange**(s):

In [ ]:
add_with_ranges.exprInfo()

For convenience, there is a `varRange` method for creating an **ExprRange** over an indexed variable.

In [ ]:
from proveit import varRange, IndexedVar
varRange(x, one, n)

Because of the `relabeling` freedom of **Lambda** expressions (alpha conversion), this is treated as `__eq__` to any similar **ExprRange** using any `parameter`.

In [ ]:
varRange(x, one, n) == ExprRange(k, IndexedVar(x, k), one, n)

In [ ]:
varRange(x, one, n) == ExprRange(a, IndexedVar(x, a), one, n)

Also note that there are a variety of variable ranges already defined for convenience in `proveit.core_expr_types._common_`:

In [ ]:
from proveit.core_expr_types._common_ import a_i_to_j, a_1_to_n, x_1_to_n

In [ ]:
ExprTuple(a_i_to_j, a_1_to_n, x_1_to_n)

### Showing the explicit parameterization

Some **ExprRanges** can be ambiguous in the default formatting style.  For example, the following expression are different but have identical default formatting

In [ ]:
range__add1__0_to_n = ExprRange(k, Add(k, one), zero, n)

In [ ]:
range__0_plus_1__to__n_plus_1 = ExprRange(k, k, Add(zero, one), Add(n, one))

In [ ]:
range__add1__0_to_n==range__0_plus_1__to__n_plus_1

We have an alternate *style* for formatting an **ExprRange** without this ambiguity.  This style is called *explicit parameterization*.  In between the first and last of the range, it shows the body of the lambda map of the **ExprRange**.

In [ ]:
range__add1__0_to_n.withExplicitParameterization()

In [ ]:
range__0_plus_1__to__n_plus_1.withExplicitParameterization()

Because it is less compact and not typically necessary, this is not the default style for LaTeX formatting.  However, it is the default for "string" formatting that gets used in error messages for the sake of clarity and making the debugging process a little easier.

### Simple repetitions

When an **ExprRange** has a lambda *body* that is independent of its parameter, it requires special formatting to avoid ambiguity.  The simple case in when the start index is $1$.

In [ ]:
ExprRange(k, Add(one, two), one, n)

Otherwise, the start and end indices will be provided when the **ExprRange** is formatted.

In [ ]:
ExprRange(k, Add(one, two), i, j)

### Nested ExprRanges

**ExprRange**s may be nested to express a range of ranges representing elements of a single **ExprTuple**.  For clarity of the expression, ellipses are doubled to express a range of ranges (or tripled for a range of ranges of ranges, etc.).

In [ ]:
from proveit.number import one
double_nested = ExprRange(i, ExprRange(j, Mult(i, j), one, n), one, n)

In [ ]:
ExprTuple(double_nested)

If $n = 3$, for example, the above **ExprTuple** could expand to $((1 \cdot 1), (1 \cdot 2), (1 \cdot 3), (2 \cdot 1), (2 \cdot 2), (2 \cdot 3), (3 \cdot 1), (3 \cdot 2), (3 \cdot 3))$.

Let's now show triple nesting for fun:

In [ ]:
triple_nested = ExprRange(i, ExprRange(j, ExprRange(k, Mult(i, j, k), 
                                                    one, n), 
                                       one, n), 
                          one, n)

In [ ]:
Add(triple_nested)

The `varRange` function may also be used to make nested ranges of indexed variables by simply supplying multiple start and end indices.

In [ ]:
x_111_to_nnn = varRange(x, (one, one, one), (n, n, n))

Note that multiple indices are implemented by nesting **IndexedVar** expressions:

In [ ]:
x_111_to_nnn.body.body.body.exprInfo()

## Expanding ExprRanges

The **ExprRange** and **IndexedVar** expression types work together in **ExprRange** expansion reduction rule.  Essentially, when a range of indexed variables is instantiated to an **ExprTuple** of elements and those indexed variables are contained within an **ExprRange**, that **ExprRange** must be "expanded" to reflect the expansion of the indexed variables.

### Parameter dependent expansions

The more restrictive form of **ExprRange** expansion is when **ExprRange** parameter is used for anything other than indices of indexed variables being expanded.  For example, consider the following lambda expression.

In [ ]:
param_dep_lambda_example = Lambda(x_1_to_n, Add(ExprRange(k, Mult(k, IndexedVar(x, k)), 
                                                          one, n)))

Note that the **ExprRange** parameter on the right appears as a prefactor as well as an index of $x$.  To respect the restrictions of a parameter dependent expansion, in this case, we may only apply this lambda map to an **ExprTuple** with **ExprRanges** whose indices match the original indices.  We demonstrate this below by applying `param_dep_lambda_example` to $a_1, \ldots, a_j, a_{j+1}, \ldots, a_n$.  First, we prove a few things in order to meet the necessary *requirements* for the expansion under a set of *assumption*

In [ ]:
from proveit.core_expr_types import Len
from proveit.logic import InSet
from proveit.number import Naturals, subtract
param_dep_assumptions = [InSet(j, Naturals), InSet(subtract(n, j), Naturals),
                         InSet(n, Naturals)]
defaults.assumptions = param_dep_assumptions

In [ ]:
a_1_to_n_partition = a_1_to_n.partition(j)

In [ ]:
typical = Len(a_1_to_n_partition.lhs).typical_equiv()

In [ ]:
from proveit.logic import Equals
equality = Equals(Len(a_1_to_n_partition.rhs), typical.rhs)

In [ ]:
Len(a_1_to_n_partition.rhs).deduceEquality(equality)

In [ ]:
typical.rhs.operand.entries[0].partition(j)

In [ ]:
param_dep_operands = a_1_to_n_partition.rhs

Now we will apply `param_dep_lambda_example` to $a_1, \ldots, a_j, a_{j+1}, \ldots, a_n$

In [ ]:
requirements = []
param_dep_lambda_example.apply(*param_dep_operands, requirements = requirements)

The following requirements are needed for this lambda application.  First, the length of the operands must match the length of the parameter index range for a proper applications (as we saw in the previous chapter).  Second, the indices themselves must match in order to perform this parameter-dependent **ExprRange** expansion.

In [ ]:
requirements

### Parameter independent expansions

When the **ExprRange** parameter only appears as indices of variables that are being expanded, the **ExprRange** expansion reduction is more permissive.  Indices do not need to match.  Consider the following `dot_prod_lambda` example.

In [ ]:
from proveit.core_expr_types._common_ import (
    a_1_to_j, b_1_to_k, c_1_to_j, d_1_to_k, x_1_to_n, y_1_to_n)
dot_prod_lambda = \
    Lambda((x_1_to_n, y_1_to_n), 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, n)))

We map expand the $x_k$ variables and $y_k$ variables in any manner, with respecting the original indices, as long as the expansions of these two match.  We will demonstrate this by applying `dot_prod_lambda` such that the $x$ operands will be $(a_1, \dots, a_j, b_1, \ldots, b_k)$ and the $y$ operands will be $(c_1, \ldots, c_j, 1 \cdot d_1, \ldots, k \cdot d_k)$.  These don't respect the original indices since they go up to $j$ and then restart at $1$, but the do match each other, both going $1$ to $j$ then $1$ to $k$.  First, we will make some assumptions and prove a few things so we can meet the requirements.

In [ ]:
jpk_eq_n = Equals(Add(j, k), n)

In [ ]:
param_indep_assumptions = [InSet(j, Naturals), InSet(k, Naturals), jpk_eq_n]
defaults.assumptions = param_indep_assumptions

In [ ]:
jpk_in_nats = InSet(Add(j, k), Naturals).prove()

In [ ]:
jpk_eq_n.subRightSideInto(jpk_in_nats)

In [ ]:
x_operands = [a_1_to_j, b_1_to_k]

Make this one a little more interesting and demonstrate that expansions can be **ExprRange**s of any kind, not just variable ranges.

In [ ]:
y_operands = [c_1_to_j, ExprRange(i, Mult(i, IndexedVar(d, i)), one, k)]

In [ ]:
x_len_jpk = Len(x_operands).computation()

In [ ]:
jpk_eq_n.subRightSideInto(x_len_jpk)

In [ ]:
y_len_jpk = Len(y_operands).computation()

In [ ]:
jpk_eq_n.subRightSideInto(y_len_jpk)

Now we perform the lambda application

In [ ]:
requirements = []
dot_prod_lambda.apply(*x_operands, *y_operands, requirements=requirements)

Note that the requirements, in this case, only relate to matching the length of the operands and respective parameter index ranges.  The necessity that the $x$ operands and the $y$ operands match with respect to corresponding **ExprRange** start and end indices is not needed as an explicit requirement; it is a simple matter of checking that the expressions of these indices match identically.  In other words, that requirement is met by a trivial inspection.

In [ ]:
requirements

Parameter independent expansions are only possible when all of the indexed variables of a particular expression range are expanded.

In [ ]:
dot_prod_with_y = \
    Lambda(x_1_to_n, 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, n)))

In [ ]:
from proveit import LambdaApplicationError
try:
    dot_prod_with_y.apply(a_1_to_j, b_1_to_k, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

But we can do a parameter dependent expansion in this case, as long as we can prove that the indices of the expansion match the original indices.

In [ ]:
requirements = []
dot_prod_with_y.apply(*param_dep_operands, assumptions=param_dep_assumptions, 
                      requirements=requirements)

In [ ]:
requirements

Also note that an explicit expansion must be provided for each precise range containing an expanded range of variables.

In [ ]:
inconsistent_dot_prod = \
    Lambda((x_1_to_n, y_1_to_n), 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, m)))

In [ ]:
try:
    inconsistent_dot_prod.apply(*x_operands, *y_operands, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

However, we may supply *equivalent alternative expansions*, `equiv_alt_expansions`, to handle such cases as appropriate.  This was covered in the previous chapter and will not be revisited here.

All of the expansions within an **ExprRange** must match with respect to the indices of their **ExprRanges**.  That means that the number of entries of the expansions must match:

In [ ]:
try:
    dot_prod_lambda.apply(a_1_to_j, b_1_to_k, y_1_to_n, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

They must match with respect to which entries are singular versus **ExprRanges**:

In [ ]:
from proveit.number import NaturalsPos
defaults.assumptions = [InSet(n, NaturalsPos)] + param_indep_assumptions

In [ ]:
x_partition = x_1_to_n.partition(one)

In [ ]:
x_operands_new = x_partition.rhs

In [ ]:
dot_prod_lambda

In [ ]:
try:
    dot_prod_lambda.apply(*x_operands_new, a_1_to_j, b_1_to_k, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Each of the start indices must match:

In [ ]:
from proveit.number import zero
x_shift_equiv = x_1_to_n.shift_equivalence(new_start=zero)

In [ ]:
x_operands_shifted = x_shift_equiv.rhs

In [ ]:
try:
    dot_prod_lambda.apply(*x_operands_shifted, y_1_to_n, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

And each of the end indices must match:

In [ ]:
defaults.assumptions = param_indep_assumptions

In [ ]:
y_operands_new = [b_1_to_k, a_1_to_j]

In [ ]:
ynew_len_kpj = Len(y_operands_new).computation()

In [ ]:
ynew_len_jpk = ynew_len_kpj.innerExpr().rhs.commute()

In [ ]:
jpk_eq_n.subRightSideInto(ynew_len_jpk)

In [ ]:
try:
    dot_prod_lambda.apply(*x_operands, *y_operands_new, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

## Nested ExprRange expansions

As we saw above, we may have nested **ExprRange** expressions and ranges of ranges of variables/parameters.

In [ ]:
x_11_to_mn = varRange(x, (one, one), (m, n))

Consider the following two different lambda expressions mapping a range of ranges of parameters.  The first multiplies groups of sums of terms and the second is a range of ranges of terms added together. 

In [ ]:
double_nested_lambda1 = Lambda(x_11_to_mn, Mult(
    ExprRange(i, Add(ExprRange(j, Mult(i, j, IndexedVar(x, (i, j))), 
                               one, n)), 
              one, m)))

In [ ]:
double_nested_lambda2 = Lambda(x_11_to_mn, Add(
    ExprRange(i, ExprRange(j, Mult(i, j, IndexedVar(x, (i, j))), 
                           one, n), 
              one, m)))

First, let's consider a simple example in which we assume that $m=2$ and $n=2$ and apply `double_nested_lambda` to $(a, b, c, d)$.

In [ ]:
abcd = ExprTuple(a, b, c, d)

We'll make some assumptions and prove a few things to meet the requirements.

In [ ]:
from proveit.number import two
defaults.assumptions = [Equals(n, two), Equals(m, two)]

In [ ]:
from proveit import extract_var_tuple_indices
Equals(Len(abcd), Len(extract_var_tuple_indices(ExprTuple(x_11_to_mn)))).prove()

In [ ]:
eq = Equals(ExprTuple(one, two), ExprTuple(ExprRange(k, k, one, two))).prove()

In [ ]:
Equals(n, two).subLeftSideInto(eq.innerExpr().rhs[0].end_index)

In [ ]:
Equals(m, two).subLeftSideInto(eq.innerExpr().rhs[0].end_index)

Now we will apply `double_nested_lambda1` to $(a, b, c, d)$ and demonstrate the expansion of nested **ExprRanges**.

In [ ]:
requirements=[]
double_nested_lambda1.apply(*abcd, requirements=requirements)

Note in the requirements below that we need indices to match, (1,2)=(1,…,𝑛)
and (1,2)=(1,…,𝑚) in order to perform this parameter-dependent expansion.

In [ ]:
# remove duplicate requirements using an OrderedDict
from collections import OrderedDict
requirements = list(OrderedDict.fromkeys(requirements))
requirements

Now we will apply `double_nested_lambda2` to $(a, b, c, d)$ and demonstrate the expansion of nested **ExprRanges**.

In [ ]:
requirements=[]
double_nested_lambda2.apply(*abcd, requirements=requirements)

Note in the requirements below that we need indices to match, $(1, 2) = (1, \ldots, n)$ and $(1, 2) = (1, \ldots, m)$ in order to perform this parameter-dependent expansion.

In [ ]:
# remove duplicate requirements using an OrderedDict
from collections import OrderedDict
requirements = list(OrderedDict.fromkeys(requirements))
requirements

Let's do one more example to test this capability

In [ ]:
double_nested_operands = ExprTuple(varRange(x, (one, one), (subtract(m, one), n)),
                                   ExprRange(k, Mult(k, k), one, n))

In [ ]:
defaults.assumptions = [InSet(m, NaturalsPos), InSet(n, Naturals)]

In [ ]:
ExprRange(k, k, one, m).partition(subtract(m, one))

In [ ]:
requirements=[]
double_nested_lambda1.apply(*double_nested_operands, requirements=requirements)

In [ ]:
requirements

In [ ]:
requirements=[]
double_nested_lambda2.apply(*double_nested_operands, requirements=requirements)

In [ ]:
requirements

### Row-major vs column-major order

By our convention we the linearize a range of ranges of parameters according to row-major ordering.  This is what we used in the example above and is what `varRange` produced.  A column-major ordering is allowed, however, as long as it is employed consistently.  We repeat one of the above examples using column-major order.

In [ ]:
x_11_to_mn__column_major = ExprRange(i, ExprRange(j, IndexedVar(x, (j, i)), one, n), one, m)

In [ ]:
double_nested_lambda1_column_major = Lambda(x_11_to_mn__column_major, Mult(
    ExprRange(i, Add(ExprRange(j, Mult(i, j, IndexedVar(x, (j, i))), 
                               one, n)), 
              one, m)))

In [ ]:
defaults.assumptions = [Equals(n, two), Equals(m, two)]

In [ ]:
requirements = []
double_nested_lambda1_column_major.apply(*abcd, requirements=requirements)

If the ordering is not consistent, such as using row-major order for the parameters but column-major order in the bodoy of the lambda expression, an error will occur.

In [ ]:
double_nested_lambda1_mixed_order = Lambda(x_11_to_mn, Mult(
    ExprRange(i, Add(ExprRange(j, Mult(i, j, IndexedVar(x, (j, i))), 
                               one, n)), 
              one, m)))

In [ ]:
try:
    requirements = []
    double_nested_lambda1_mixed_order.apply(*abcd, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

## Singular and empty range reductions

When the length of an **ExprRange** is known to be empty or singular, in the process of performing a lambda application, the **ExprRange** will be eliminated appropriately within the **ExprTuple** in which it is contained.  For example,

In [ ]:
tuple_with_range_lambda = Lambda(n, Add(a, ExprRange(i, Mult(i, i), one, n), c))

Applying this lambda to set $n : 0$ will eliminate this **ExprRange**, leaving just the $a$ and $c$ terms in this sum.

In [ ]:
requirements = []
tuple_with_range_lambda.apply(zero, requirements=requirements)

The *requirement* for this reduction is a statement that the **ExprTuple** of the **ExprRange**, with $n$ replaced by $0$, is equal to the **ExprTuple**.  Note that this is not effecting a direct substitution of these **ExprTuple**s since the original **ExprRange** is embedded in an **ExprTuple** with other entries.  So this reduction requires special treatment compared with the *equivalence reductions* that will be discussed in a later chapter.

In [ ]:
requirements

This reduction works as long as it is known that the `end_index + 1 = start index`.

In [ ]:
tuple_with_ij_range_lambda = Lambda(j, Add(a, ExprRange(b, Mult(b, b), i, j), c))

In [ ]:
Add(subtract(i, one), one).simplification([InSet(i, Naturals)])

In [ ]:
requirements = []
tuple_with_ij_range_lambda.apply(subtract(i, one), assumptions=[InSet(i, Naturals)],
                                 requirements=requirements)

In [ ]:
requirements

To reduce the **ExprRange** to a singular element, the `start_index` and `end_index` must match exactly.

In [ ]:
requirements = []
tuple_with_ij_range_lambda.apply(i, requirements=requirements)

In [ ]:
requirements

There are no other special **ExprRange** reductions, even if the expansion is obvious.  For example,

In [ ]:
from proveit.number import two
tuple_with_range_lambda.apply(two)

Depending upon the circumstance, making the explicit expansion may not be desirable.  Such replacements should be made explicitly via the **theory system** rather than complicating the **core derivation system**.

In [ ]:
%end exprrange_reductions

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>